## CS-323 Artificial Intelligence | Complex Engineering Problem
### TOPIC:
##### **"Implementation and Comparative Analysis of AI Search Techniques for Solving the 8-Quuens Problem"**

**Group Members:** 
- Areeba Khan CS23110
- Zara Akram CS23104
  

## Problem Statement

**N-Queens** is the challenge of placing N queens on an N×N chessboard so that none attack each other.  
This is a classic combinatorial problem that demonstrates constraint satisfaction and search techniques in AI.

---

In [1]:
import random, time, heapq, csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
from datetime import datetime

random.seed(42)
np.random.seed(42)
print("✓ Setup complete")

✓ Setup complete


In [2]:
# ==================== HELPERS FUNCTIONS ====================
def count_conflicts(state):
    conflicts = 0
    n = len(state)
    for i in range(n):
        for j in range(i + 1, n):
            if state[i] == state[j] or abs(state[i] - state[j]) == abs(i - j):
                conflicts += 1
    return conflicts

def get_attacking_pairs(state):
    pairs = []
    n = len(state)
    for i in range(n):
        for j in range(i + 1, n):
            if state[i] == state[j] or abs(state[i] - state[j]) == abs(i - j):
                pairs.append((i, state[i], j, state[j]))
    return pairs

def is_solution(state):
    return count_conflicts(state) == 0

def random_state(n=8):
    return [random.randint(0, n-1) for _ in range(n)]

def get_neighbors(state):
    neighbors = []
    n = len(state)
    for col in range(n):
        for row in range(n):
            if row != state[col]:
                new_state = state.copy()
                new_state[col] = row
                neighbors.append(new_state)
    return neighbors

print("✓ Helpers Functions ready")

✓ Helpers Functions ready


In [3]:
# ==================== BOARD VISUALIZATION ====================
def draw_boards(states, titles, show_conflicts_list=None, size=3.5, star_size=20, state_arrays=None, conflicts_list=None, labels=None, state_colors=None):
    """
    Draw boards side by side; state arrays are placed ABOVE each board with matching colors and alignment.
    """
    n_boards = len(states)
    fig, axes = plt.subplots(1, n_boards, figsize=(size*n_boards, size))
    if n_boards == 1:
        axes = [axes]
    if show_conflicts_list is None:
        show_conflicts_list = [False] * n_boards
    if state_arrays is None:
        state_arrays = states
    if conflicts_list is None:
        conflicts_list = [count_conflicts(s) if s else 0 for s in states]
    if labels is None:
        labels = titles
    if state_colors is None:
        # Pick nice colors for each state
        state_colors = ["#e74c3c", "#27ae60", "#27ae60"] + ["#2980b9"] * (n_boards - 3)

    for idx, (ax, state, title, show_conf, arr, conf, label, color) in enumerate(zip(
            axes, states, titles, show_conflicts_list, state_arrays, conflicts_list, labels, state_colors)):
        n = len(state) if state else 8
        # Draw board
        for row in range(n):
            for col in range(n):
                color_sq = '#F0D9B5' if (row + col) % 2 == 0 else '#B58863'
                rect = patches.Rectangle((col, row), 1, 1, facecolor=color_sq,
                                         edgecolor='#8B7355', linewidth=1.2)
                ax.add_patch(rect)
        # Row/Col numbers
        ax.set_xticks([i + 0.5 for i in range(n)])
        ax.set_yticks([i + 0.5 for i in range(n)])
        ax.set_xticklabels(range(1, n + 1), fontsize=7, fontweight='bold')
        ax.set_yticklabels(range(1, n + 1), fontsize=7, fontweight='bold')
        ax.xaxis.tick_top()
        ax.yaxis.tick_left()
        ax.tick_params(axis='both', which='both', length=0)
        # Conflict lines
        if state and show_conf:
            for col1, row1, col2, row2 in get_attacking_pairs(state):
                ax.plot([col1 + 0.5, col2 + 0.5], [row1 + 0.5, row2 + 0.5],
                        'r-', linewidth=2, alpha=0.7, zorder=1)
        # Queens with size
        if state:
            for col, row in enumerate(state):
                ax.plot(col + 0.5, row + 0.5, marker='*', markersize=star_size,
                        color='#8B0000', markeredgecolor='#FFD700', markeredgewidth=2, zorder=2)
        ax.set_xlim(0, n)
        ax.set_ylim(0, n)
        ax.set_aspect('equal')
        ax.invert_yaxis()
        conf_text = f"{conf} Conflicts" if conf > 0 else "Valid"
        title_color = '#27ae60' if conf == 0 else '#e74c3c'
        # State array goes ABOVE, with bold color
        arr_txt = "[" + ", ".join(str(x) if x != -1 else '-' for x in arr) + "]"
        ax.set_title(f"{label}\n{arr_txt}\n{conf_text}", fontsize=10, weight='bold', color=title_color, pad=9)
    plt.tight_layout()
    plt.show()
print("✓ Board visuals rendered")

✓ Board visuals rendered


### Method 1: A* Search
A* is a widely used informed search technique. Here, we use it to find a path from a random (possibly illegal) starting arrangement to a valid N-Queen configuration.

- **State:** board arrangement (one queen per column)
- **Heuristic:** number of conflicting queen pairs (minimum = 0)
- **Goal Test:** zero conflicts

### Implementation Highlights
- Each move modifies one queen's position.
- Always expands the board with the fewest conflicts.
- Finds a solution if one is reachable from the start.
---

In [4]:
def astar_search(initial_state, max_iterations=100000):
    start_time = time.time()
    open_list = []
    closed_set = set()
    
    g = 0
    h = count_conflicts(initial_state)
    heapq.heappush(open_list, (g+h, g, 0, tuple(initial_state)))
    
    nodes_expanded = 0
    max_frontier = 1
    counter = 1
    
    while open_list and nodes_expanded < max_iterations:
        f, g, _, current_tuple = heapq.heappop(open_list)
        current_state = list(current_tuple)
        
        if current_tuple in closed_set:
            continue
        
        closed_set.add(current_tuple)
        nodes_expanded += 1
        
        if is_solution(current_state):
            return current_state, {
                'success': True, 'nodes': nodes_expanded,
                'time': time.time() - start_time, 'path_length': g,
                'max_frontier': max_frontier, 'final_conflicts': 0
            }
        
        for neighbor in get_neighbors(current_state):
            neighbor_tuple = tuple(neighbor)
            if neighbor_tuple not in closed_set:
                g_new = g + 1
                h_new = count_conflicts(neighbor)
                heapq.heappush(open_list, (g_new+h_new, g_new, counter, neighbor_tuple))
                counter += 1
        
        max_frontier = max(max_frontier, len(open_list))
    
    return current_state, {
        'success': False, 'nodes': nodes_expanded,
        'time': time.time() - start_time, 'path_length': None,
        'max_frontier': max_frontier,
        'final_conflicts': count_conflicts(current_state)
    }
print("✓ A* algorithm loaded")
print(nodes_expanded)
    
    



✓ A* algorithm loaded


NameError: name 'nodes_expanded' is not defined

### Method 2: CSP Backtracking
The N-Queens problem naturally fits, a constraint satisfaction problem. CSP exploits this by only exploring “legal” placements column-by-column.

- **Heuristics:** MRV (Minimum Remaining Values), Forward Checking
- **Guarantees:** Finds a solution or reports none
- **Backtracks:** If it reaches a dead end, it tries the next best option

### Key CSP Steps
- Place queens one column at a time.
- Use forward checking to prune impossible options.
- Track number of backtracks for efficiency comparison
---


In [ ]:
def is_consistent(var, value, assignment):
    for col, row in assignment.items():
        if row == value or abs(row - value) == abs(col - var):
            return False
    return True

def forward_check(var, value, domains, n):
    removed = {}
    for future_col in range(var + 1, n):
        removed[future_col] = set()
        for row in list(domains[future_col]):
            if row == value or abs(row - value) == abs(future_col - var):
                domains[future_col].discard(row)
                removed[future_col].add(row)
        if not domains[future_col]:
            return None
    return removed

def backtrack_csp(assignment, domains, n, metrics, track_steps=False, steps_list=None):
    metrics['calls'] += 1
    
    if len(assignment) == n:
        return assignment
    
    if track_steps and steps_list is not None:
        current_state = [-1] * n
        for col, row in assignment.items():
            current_state[col] = row
        steps_list.append({'state': current_state.copy(), 'step': len(assignment)})
    
    var = min([c for c in range(n) if c not in assignment], key=lambda c: len(domains[c]))
    metrics['nodes'] += 1
    
    for value in list(domains[var]):
        if is_consistent(var, value, assignment):
            assignment[var] = value
            domains_backup = {k: v.copy() for k, v in domains.items()}
            
            removed = forward_check(var, value, domains, n)
            if removed is not None:
                result = backtrack_csp(assignment, domains, n, metrics, track_steps, steps_list)
                if result is not None:
                    return result
            
            del assignment[var]
            domains.update(domains_backup)
            metrics['backtracks'] += 1
    
    return None

def solve_csp(n=8, track_steps=False):
    start_time = time.time()
    domains = {col: set(range(n)) for col in range(n)}
    assignment = {}
    metrics = {'nodes': 0, 'calls': 0, 'backtracks': 0}
    steps_list = [] if track_steps else None
    
    result = backtrack_csp(assignment, domains, n, metrics, track_steps, steps_list)
    
    if result:
        solution = [result[col] for col in range(n)]
        return solution, {
            'success': True, 'nodes': metrics['nodes'],
            'backtracks': metrics['backtracks'],
            'time': time.time() - start_time, 'steps': steps_list
        }
    return None, {
        'success': False, 'nodes': metrics['nodes'],
        'backtracks': metrics['backtracks'],
        'time': time.time() - start_time
    }

print("✓ CSP Algorithms ready")

In [ ]:
# ==================== GLOBAL STATE + CSV EXPORT FUNCTINALITY ====================
current_state = None
astar_ran = False
csp_ran = False
astar_result = None
csp_result = None
csp_stages = []

# CSV Export functionality
results_log = []

def save_to_csv():
    """Save all results to CSV"""
    if not results_log:
        print("No results to save yet!")
        return
    
    filename = f'8queens_results_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
    df = pd.DataFrame(results_log)
    df.to_csv(filename, index=False)
    print(f"✓ Results saved to: {filename}")
    return filename

def log_result(algo, n, initial, solution, metrics, stages=None):
    """Only log summary results for CSV (no per-stage columns)"""
    entry = {
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'algorithm': algo,
        'board_size': n,
        'initial_state': str(initial) if initial else 'N/A',
        'solution_state': str(solution),
        'success': metrics['success'],
        'time_seconds': metrics['time'],
        'nodes_expanded': metrics['nodes'],
        'path_length': metrics.get('path_length', ''),
        'backtracks': metrics.get('backtracks', ''),
        'final_conflicts': metrics.get('final_conflicts', count_conflicts(solution) if solution else 0)
    }
    results_log.append(entry)

# Output area
output_area = widgets.Output()

print("✓ CSV export system ready")

## Algorithm Comparison

After running both algorithms, use **Compare** to see side-by-side:
- Initial state vs. A* solution vs. CSP solution
- Key metrics (time, nodes, conflicts, etc.)
- Instant graphical summary (bar charts for speed, nodes, and steps/backtracks)

---

In [ ]:
# ==================== Comparison Table Function ====================
def show_enhanced_table(astar_met, csp_met, n, init_state, astar_sol, csp_sol):
    """Slim table with just overall summary (no CSP stages, only summary)."""
    return f"""
    <style>
        .enh-table {{
            width:100%; border-collapse:collapse; margin:15px 0;
            box-shadow:0 4px 12px rgba(0,0,0,0.15); border-radius:10px; overflow:hidden;
        }}
        .enh-table th {{
            background:linear-gradient(135deg,#667eea,#764ba2);
            color:white; padding:12px; font-size:13px; font-weight:bold;
        }}
        .enh-table td {{
            padding:10px; border-bottom:1px solid #ddd; font-size:12px;
        }}
        .enh-table tr:nth-child(even) {{ background:#f8f9fa; }}
        .enh-table tr:hover {{ background:#e3f2fd; }}
        .sol {{
            font-family:monospace; background:#f0f0f0;
            padding:4px 8px; border-radius:4px; font-size:13px;
        }}
        .metric {{ font-weight:600; color:#2c3e50; }}
    </style>
    <table class="enh-table">
        <thead>
            <tr><th>Metric</th><th>A* Search</th><th>CSP Backtracking</th></tr>
        </thead>
        <tbody>
            <tr>
                <td class="metric">Board Size</td>
                <td colspan="2" style="text-align:center;"><b>{n}×{n}</b></td>
            </tr>
            <tr>
                <td class="metric">Generated/Initial State</td>
                <td class="sol">{init_state}<br><small>{count_conflicts(init_state)} conflicts</small></td>
                <td class="sol">Empty board<br><small>0 conflicts</small></td>
            </tr>
            <tr>
                <td class="metric">Final Solution</td>
                <td class="sol">{astar_sol}<br><small>{count_conflicts(astar_sol)} conflicts</small></td>
                <td class="sol">{csp_sol}<br><small>{count_conflicts(csp_sol)} conflicts, {csp_met['nodes']} steps</small></td>
            </tr>
            <tr>
                <td class="metric">Execution Time</td>
                <td>{astar_met['time']:.6f}s</td>
                <td>{csp_met['time']:.6f}s</td>
            </tr>
            <tr>
                <td class="metric">Nodes Expanded</td>
                <td>{astar_met['nodes']:,}</td>
                <td>{csp_met['nodes']:,}</td>
            </tr>
            <tr>
                <td class="metric">Path Length / Backtracks</td>
                <td>{astar_met.get('path_length','N/A')}</td>
                <td>{csp_met['backtracks']}</td>
            </tr>
        </tbody>
    </table>
    """


In [ ]:
# ==================== Comparison Graphs + Button Handlers ====================

def show_comparison_graphs(astar_met, csp_met, astar_sol, csp_sol):
    fig, axs = plt.subplots(1, 3, figsize=(15, 4.3))
    labels = ["A*", "CSP"]

    # 1. Execution Time
    times = [astar_met['time'], csp_met['time']]
    axs[0].bar(labels, times, color=['#3498db','#27ae60'])
    axs[0].set_title("Execution Time (s)")
    axs[0].set_ylabel("Time (seconds)")
    for i, v in enumerate(times):
        axs[0].text(i, v, f"{v:.4f}", ha='center', va='bottom', fontsize=11, weight='bold')

    # 2. Nodes Expanded
    nodes = [astar_met['nodes'], csp_met['nodes']]
    axs[1].bar(labels, nodes, color=['#3498db','#27ae60'])
    axs[1].set_title("Nodes Expanded")
    axs[1].set_ylabel("Nodes")
    for i, v in enumerate(nodes):
        axs[1].text(i, v, f"{v}", ha='center', va='bottom', fontsize=11, weight='bold')

    # 3. Path/Backtracks Comparison
    astar_path = astar_met.get('path_length', 0) or 0
    csp_backtracks = csp_met.get('backtracks', 0)
    axs[2].bar(["A* Path Length", "CSP Backtracks"], [astar_path, csp_backtracks], color=['#e67e22','#e67e22'])
    axs[2].set_title("Path Length vs Backtracks")
    axs[2].set_ylabel("Steps")
    for i, v in enumerate([astar_path, csp_backtracks]):
        axs[2].text(i, v, f"{int(v)}", ha='center', va='bottom', fontsize=11, weight='bold')

    fig.suptitle("Algorithm Comparison", fontsize=14, weight='bold')
    plt.tight_layout()
    plt.show()
    display(HTML("<b>Interpretation:</b> These graphs compare both algorithms for this run's state: which is faster, more efficient, and which needed more search steps."))

# ==================== BUTTON HANDLERS ====================
def on_generate(b):
    global current_state, astar_ran, csp_ran, csp_stages
    n = n_slider.value
    current_state = random_state(n)
    astar_ran = csp_ran = False
    csp_stages = []
    with output_area:
        clear_output(wait=True)
        display(HTML("""
        <div style='background:linear-gradient(135deg,#3498db,#2980b9);padding:15px;
                    border-radius:8px;color:white;margin:10px 0;text-align:center;'>
            <h3 style='margin:0;'>State Generated for A*</h3>
        </div>
        """))
        draw_boards([current_state], ["Generated State"], [True], size=4, star_size=25)

def on_run_astar(b):
    global astar_ran, astar_result
    if current_state is None:
        with output_area:
            clear_output()
            display(HTML("""
            <div style='background:#fff3cd;padding:15px;border-radius:8px;
                        border-left:5px solid #ffc107;margin:10px 0;'>
                <h4 style='margin:0;color:#856404;'>⚠️ Warning</h4>
                <p style='margin:5px 0 0 0;color:#856404;'>Please generate a state first!</p>
            </div>
            """))
        return
    with output_area:
        clear_output(wait=True)
        display(HTML("""
        <div style='background:linear-gradient(135deg,#3498db,#2980b9);padding:15px;
                    border-radius:8px;color:white;margin:10px 0;text-align:center;'>
            <h3 style='margin:0;'>A* Search Running...</h3>
        </div>
        """))
        solution, metrics = astar_search(current_state, max_iterations=100000)
        astar_result = (solution, metrics)
        astar_ran = True
        log_result('A*', len(current_state), current_state, solution, metrics)
        draw_boards([current_state, solution], ["A*: Initial", "A*: Solution"], [True, True], size=4, star_size=25)
        display(HTML(f"""
        <div style='background:#e3f2fd;padding:15px;border-radius:8px;margin:10px 0;'>
            <h4 style='color:#1976d2;margin:0 0 10px 0;'>A* Results</h4>
            <p><b>Solution:</b> <code style='background:#fff;padding:3px 8px;border-radius:3px;'>{solution}</code></p>
            <p><b>Status:</b> <span style='color:{'#27ae60' if metrics['success'] else '#e74c3c'};font-weight:bold;'>
               {'SUCCESS ✓' if metrics['success'] else 'FAILED ✗'}</span></p>
            <p><b>Time:</b> {metrics['time']:.6f}s | 
               <b>Nodes:</b> {metrics['nodes']:,} | 
               <b>Path:</b> {metrics.get('path_length','N/A')}</p>
        </div>
        """))

def on_run_csp(b):
    global csp_ran, csp_result, csp_stages
    n = n_slider.value
    with output_area:
        clear_output(wait=True)
        display(HTML("""
        <div style='background:linear-gradient(135deg,#27ae60,#229954);padding:15px;
                    border-radius:8px;color:white;margin:10px 0;text-align:center;'>
            <h3 style='margin:0;'>CSP Backtracking Running...</h3>
        </div>
        """))
        solution, metrics = solve_csp(n, track_steps=True)
        csp_result = (solution, metrics)
        csp_ran = True
        csp_stages = metrics['steps'] if metrics.get('steps') else []
        log_result('CSP', n, None, solution, metrics, csp_stages)
        # Progression boards code (as above)
        filtered = []
        prev_q = -1
        for step in csp_stages:
            count = sum(1 for s in step['state'] if s >= 0)
            if count > prev_q:  # only steps where a new queen is placed
                filtered.append(step)
                prev_q = count
        main_steps = []
        indices_to_show = [0, 1]
        board_steps = [int(n/4), int(n/2), int(3*n/4), n]
        for v in board_steps:
            idx = next((i for i, st in enumerate(filtered) if sum(1 for x in st['state'] if x >= 0) == v), None)
            if idx is not None:
                indices_to_show.append(idx)
        indices_to_show = [i for i in indices_to_show if i < len(filtered)]
        key_steps = [filtered[i] for i in sorted(set(indices_to_show))]
        if len(key_steps) < 2:
            key_steps = filtered[:max(2, len(filtered))]
        states_show = []
        titles_show = []
        conflicts_show = []
        for i, step in enumerate(key_steps):
            arr = step['state']
            n_queens = sum(1 for x in arr if x >= 0)
            if n_queens == 0:
                label = "Empty"
            else:
                label = f"{n_queens}/{n} Queens Placed"
            percent = int(round((n_queens / n) * 100))
            step_title = f"{label} ({percent}%)"
            titles_show.append(step_title)
            states_show.append([(x if x >= 0 else -1) for x in arr])
            conflicts_show.append(True)
        draw_boards(states_show, titles_show, conflicts_show, size=3, star_size=15)
        draw_boards([solution], ["CSP Solution"], [False], size=4, star_size=25)
        totals = f"<p><b>Final Solution Steps:</b> {metrics['nodes']} steps | <b>Backtracks:</b> {metrics['backtracks']}</p>"
        display(HTML(f"""
        <div style='background:#e8f5e9;padding:12px;border-radius:8px;margin:8px 0;'>
            <h4 style='color:#2e7d32;margin:0 0 6px 0;'>CSP Results</h4>
            <p><b>Solution:</b> <code style='background:#fff;padding:3px 8px;border-radius:3px;'>{solution}</code></p>
            <p><b>Status:</b> <span style='color:#27ae60;font-weight:bold;'>SUCCESS ✓</span></p>
            <p><b>Time:</b> {metrics['time']:.6f}s | 
               <b>Nodes:</b> {metrics['nodes']:,} | 
               <b>Backtracks:</b> {metrics['backtracks']}</p>
            {totals}
        </div>
        """))

def on_compare(b):
    global csp_stages
    if not astar_ran or not csp_ran:
        with output_area:
            clear_output()
            msg = []
            if not astar_ran:
                msg.append("Run A* first")
            if not csp_ran:
                msg.append("Run CSP first")
            display(HTML(f"""
            <div style='background:#fff3cd;padding:20px;border-radius:10px;
                        border-left:5px solid #ffc107;margin:20px 0;text-align:center;'>
                <h3 style='margin:0;color:#856404;'>⚠️ Cannot Compare Yet</h3>
                <p style='margin:10px 0;color:#856404;font-size:14px;'>
                    Please <b>{' and '.join(msg)}</b> before comparing.
                </p>
            </div>
            """))
        return
    with output_area:
        clear_output(wait=True)
        display(HTML("""
        <div style='background:linear-gradient(135deg,#9b59b6,#8e44ad);padding:15px;
                    border-radius:8px;color:white;margin:10px 0;text-align:center;'>
            <h3 style='margin:0;'>Algorithm Comparison</h3>
        </div>
        """))
        astar_sol, astar_met = astar_result
        csp_sol, csp_met = csp_result
        draw_boards(
            [current_state, astar_sol, csp_sol],
            ["Initial State", "A* Solution", "CSP Solution"],
            [True, True, False], size=3.5, star_size=22,
            state_arrays=[current_state, astar_sol, csp_sol],
            conflicts_list=[
                count_conflicts(current_state),
                count_conflicts(astar_sol),
                count_conflicts(csp_sol)
            ],
            labels=["Initial", "A* Solution", "CSP Solution"]
        )
        html = show_enhanced_table(astar_met, csp_met, n_slider.value, 
                                  current_state, astar_sol, csp_sol)
        display(HTML(html))
        show_comparison_graphs(astar_met, csp_met, astar_sol, csp_sol)
        winner = "CSP" if csp_met['time'] < astar_met['time'] else "A*"
        winner_time = min(astar_met['time'], csp_met['time'])
        display(HTML(f"""
            <div style='display:flex;justify-content:center;align-items:center;flex-direction:column;
             background:linear-gradient(90deg,#f39c12,#e67e22 100%);
             border-radius:12px;margin:18px auto 22px auto;
             padding:28px 0 22px 0;max-width:600px;box-shadow:0 2px 10px #0002'>
            <span style='font-size:29px;font-weight:900;letter-spacing:0.5px;'>
            🏆 <span style="background:#2563eb;color:#fff;padding:6px 18px 7px 18px;
            border-radius:6px;box-shadow:0 1px 6px #0002;display:inline-block;
            font-weight:800;font-size:23px;font-family:inherit;">
            Faster Algorithm: {winner}
              </span>
            </span>
        <span style='display:block;font-size:19px;color:#fff;font-weight:bold;
                 margin-top:17px;letter-spacing:0.2px;'>
        Completed in <span style="background:#ffffffcc;padding:3px 12px;
        color:#222;border-radius:4px;font-family:monospace">{winner_time:.6f} seconds</span>
    </span>
</div>
"""))

print("✓ Button handlers ready")

def on_reset(b):
    global current_state, astar_ran, csp_ran, astar_result, csp_result, csp_stages
    current_state = None
    astar_ran = csp_ran = False
    astar_result = csp_result = None
    csp_stages = []
    with output_area:
        clear_output()
        display(HTML("""
        <div style='background:linear-gradient(135deg,#95a5a6,#7f8c8d);padding:15px;
                    border-radius:8px;color:white;margin:10px 0;text-align:center;'>
            <h3 style='margin:0;'>🔄 Reset Complete</h3>
            <p style='margin:5px 0;'>Ready for new experiment</p>
        </div>
        """))

def on_save_csv(b):
    with output_area:
        filename = save_to_csv()
        if filename:
            display(HTML(f"""
            <div style='background:#d4edda;padding:15px;border-radius:8px;
                        border-left:5px solid #28a745;margin:10px 0;'>
                <h4 style='margin:0;color:#155724;'>✓ Saved to CSV</h4>
                <p style='margin:5px 0 0 0;color:#155724;'>File: {filename}</p>
            </div>
            """))


## Interactive Controls & Experiments

- Choose the board size, generate starting states, and solve using either A* or CSP or both.  
- All results and visualizations will appear below. We can compare and analyze results interactively.

---


In [ ]:
# Interactive UI display with controls and output panel

n_slider = widgets.IntSlider(
    value=8, min=4, max=12, step=1,
    description='',
    style={'description_width': '0px'},
    layout=widgets.Layout(width='300px')
)

slider_label = widgets.HTML(
    "<div style='text-align:center;margin-bottom:5px;'>"
    "<b style='font-size:14px;color:#2c3e50;'>Board Size</b></div>"
)

slider_box = widgets.VBox([slider_label, n_slider], layout=widgets.Layout(align_items='center'))

btn_gen = widgets.Button(
    description='Generate State', button_style='info', icon='refresh',
    layout=widgets.Layout(width='140px', height='35px')
)
btn_astar = widgets.Button(
    description='Run A*', button_style='primary', icon='play',
    layout=widgets.Layout(width='120px', height='35px')
)
btn_csp = widgets.Button(
    description='Run CSP', button_style='success', icon='play',
    layout=widgets.Layout(width='120px', height='35px')
)
btn_compare = widgets.Button(
    description='Compare', button_style='warning', icon='exchange',
    layout=widgets.Layout(width='120px', height='35px')
)
btn_reset = widgets.Button(
    description='Reset', button_style='', icon='undo',
    layout=widgets.Layout(width='100px', height='35px')
)
btn_csv = widgets.Button(
    description='Save CSV', button_style='success', icon='download',
    layout=widgets.Layout(width='120px', height='35px')
)

btn_gen.on_click(on_generate)
btn_astar.on_click(on_run_astar)
btn_csp.on_click(on_run_csp)
btn_compare.on_click(on_compare)
btn_reset.on_click(on_reset)
btn_csv.on_click(on_save_csv)

header = widgets.HTML("""
<div style='background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);
            padding:25px;border-radius:12px;text-align:center;color:white;
            margin:0 0 20px 0;box-shadow:0 4px 15px rgba(0,0,0,0.2);'>
    <h1 style='margin:0;font-size:28px;'>🎮 8-Queens Interactive Solver</h1>
    <p style='margin:8px 0 0 0;font-size:14px;opacity:0.9;'>
        All controls, visualizations, and results in ONE place
    </p>
</div>
""")

control_panel = widgets.VBox([
    widgets.HTML("<div style='text-align:center;margin-bottom:15px;'>"
                "<h3 style='margin:0;color:#2c3e50;'>Control Panel</h3></div>"),
    slider_box,
    widgets.HTML("<div style='height:15px;'></div>"),
    widgets.HBox([btn_gen, btn_astar, btn_csp], layout=widgets.Layout(justify_content='center')),
    widgets.HTML("<div style='height:10px;'></div>"),
    widgets.HBox([btn_compare, btn_reset, btn_csv], layout=widgets.Layout(justify_content='center'))
], layout=widgets.Layout(
    border='2px solid #ddd',
    border_radius='10px',
    padding='20px',
    margin='0 0 20px 0',
    background='#f8f9fa'
))

main_ui = widgets.VBox([
    header,
    control_panel,
    widgets.HTML("<hr style='margin:20px 0;border:none;border-top:2px solid #e0e0e0;'>"),
    output_area
])

display(main_ui)

print("✓ Main UI loaded")

## Conclusion
### Visualizations & Results Explanation

- The board below shows each queen (star), and any red lines indicate attacking pairs.
- Conflicts: Number of direct queen attacks in current state.
- "Nodes Expanded": How many configurations the algorithm searched.

---

### About Graphs

- **Execution Time:** Lower is better. Shows real speed difference.
- **Nodes Expanded:** Fewer nodes means better algorithmic pruning.
- **Path vs. Backtracks:** "A* Path Length" is how many steps were needed for a solution; "CSP Backtracks" counts restarts after dead ends—a key CSP performance indicator.

The ideal result is low time, low nodes, and low backtracks/conflicts.

---

**A* Search:**
- Variable success rate (depends on initial state)
- Finds shortest transformation path
- Higher node expansion

### Why CSP Performs Better

1. **Natural Fit:** N-Queens is inherently a constraint problem
2. **Smart Pruning:** Forward checking eliminates invalid values early
3. **MRV Heuristic:** Chooses most constrained columns first
4. **Completeness:** Guaranteed to find solution
   
### For A*
- A* finds solutions if reachable but may get stuck or be inefficient for large boards or bad starts.

---

- This Project shows the value of using algorithmic techniques that match the nature of the problem (CSP for N-Queens).